### Try k-nearest,DecisionTreeClassifier and RandomForestClassifier on below Dataset:
- 1] titanic.csv



### Problem Statement
- Predict if a passenger will survive or not.

### import required packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### get the data

In [2]:
df = pd.read_csv('titanic.csv')
df


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,14.5000,1,0,2665,14.4542,NaN,C,NaN,328.0,NaN
1305,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3,0,"Zakarian, Mr. Mapriededer",male,26.5000,0,0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3,0,"Zakarian, Mr. Ortin",male,27.0000,0,0,2670,7.2250,NaN,C,NaN,NaN,NaN


### EDA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   int64  
 1   survived   1309 non-null   int64  
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   int64  
 6   parch      1309 non-null   int64  
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [4]:
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

### Checking NA values

In [5]:
df.isna().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [6]:
per_na_values = df.isna().sum() * 100 / len(df)
per_na_values

pclass        0.000000
survived      0.000000
name          0.000000
sex           0.000000
age          20.091673
sibsp         0.000000
parch         0.000000
ticket        0.000000
fare          0.076394
cabin        77.463713
embarked      0.152788
boat         62.872422
body         90.756303
home.dest    43.086325
dtype: float64

In [7]:
drop_columns = []
threshold = 10
cols = df.columns
index = 0
for value in per_na_values:
    if abs(value) > threshold:
        drop_columns.append(cols[index])
    index += 1
drop_columns

['age', 'cabin', 'boat', 'body', 'home.dest']

In [8]:
df.drop(drop_columns, inplace=True, axis=1)
df

,pclass,survived,name,sex,sibsp,parch,ticket,fare,embarked
0,1,1,"Allen, Miss. Elisabeth Walton",female,0,0,24160,211.3375,S
1,1,1,"Allison, Master. Hudson Trevor",male,1,2,113781,151.5500,S
2,1,0,"Allison, Miss. Helen Loraine",female,1,2,113781,151.5500,S
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,1,2,113781,151.5500,S
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,1,2,113781,151.5500,S
...,...,...,...,...,...,...,...,...,...
1304,3,0,"Zabour, Miss. Hileni",female,1,0,2665,14.4542,C
1305,3,0,"Zabour, Miss. Thamine",female,1,0,2665,14.4542,C
1306,3,0,"Zakarian, Mr. Mapriededer",male,0,0,2656,7.2250,C
1307,3,0,"Zakarian, Mr. Ortin",male,0,0,2670,7.2250,C


In [9]:
df.nunique()

pclass         3
survived       2
name        1307
sex            2
sibsp          7
parch          8
ticket       929
fare         281
embarked       3
dtype: int64

- Name doesn't matter in this case (maybe if person is part of nobility has higher chance to live and   his noble status will show through his name)
- ticket number also doesn't matter much as we are taking account of class of seats

In [10]:
df.drop(['name', 'ticket'], inplace=True, axis=1)
df

,pclass,survived,sex,sibsp,parch,fare,embarked
0,1,1,female,0,0,211.3375,S
1,1,1,male,1,2,151.5500,S
2,1,0,female,1,2,151.5500,S
3,1,0,male,1,2,151.5500,S
4,1,0,female,1,2,151.5500,S
...,...,...,...,...,...,...,...
1304,3,0,female,1,0,14.4542,C
1305,3,0,female,1,0,14.4542,C
1306,3,0,male,0,0,7.2250,C
1307,3,0,male,0,0,7.2250,C


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   sibsp     1309 non-null   int64  
 4   parch     1309 non-null   int64  
 5   fare      1308 non-null   float64
 6   embarked  1307 non-null   object 
dtypes: float64(1), int64(4), object(2)
memory usage: 71.7+ KB


In [12]:
from scipy.stats import mode
df['embarked'].fillna(mode(df['embarked']).mode[0], inplace=True)
df['fare'].fillna(mode(df['fare']).mode[0], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   sibsp     1309 non-null   int64  
 4   parch     1309 non-null   int64  
 5   fare      1309 non-null   float64
 6   embarked  1309 non-null   object 
dtypes: float64(1), int64(4), object(2)
memory usage: 71.7+ KB


In [14]:
print(f"pclass categories:\n {df.pclass.unique()}\n")
# print(f"survived categories:\n {df.survived.unique()}\n")
print(f"sex categories:\n {df.sex.unique()}\n")
print(f"sibsp categories:\n {df.sibsp.unique()}\n")
print(f"parch categories:\n {df.parch.unique()}\n")
# print(f"fare categories:\n {df.fare.unique()}\n")
print(f"embarked categories:\n {df.embarked.unique()}\n")

pclass categories:
 [1 2 3]

sex categories:
 ['female' 'male']

sibsp categories:
 [0 1 2 3 4 5 8]

parch categories:
 [0 2 1 4 3 5 6 9]

embarked categories:
 ['S' 'C' 'Q']



In [15]:
df['sex'] = df['sex'].astype('category')
df['embarked'] = df['embarked'].astype('category')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   pclass    1309 non-null   int64   
 1   survived  1309 non-null   int64   
 2   sex       1309 non-null   category
 3   sibsp     1309 non-null   int64   
 4   parch     1309 non-null   int64   
 5   fare      1309 non-null   float64 
 6   embarked  1309 non-null   category
dtypes: category(2), float64(1), int64(4)
memory usage: 54.1 KB


In [17]:
from sklearn.preprocessing import LabelEncoder
df['sex'] = LabelEncoder().fit_transform(df['sex'])
df['embarked'] = LabelEncoder().fit_transform(df['embarked'])

In [18]:
df.head()

,pclass,survived,sex,sibsp,parch,fare,embarked
0,1,1,0,0,0,211.3375,2
1,1,1,1,1,2,151.5500,2
2,1,0,0,1,2,151.5500,2
3,1,0,1,1,2,151.5500,2
4,1,0,0,1,2,151.5500,2


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   int32  
 3   sibsp     1309 non-null   int64  
 4   parch     1309 non-null   int64  
 5   fare      1309 non-null   float64
 6   embarked  1309 non-null   int32  
dtypes: float64(1), int32(2), int64(4)
memory usage: 61.5 KB


In [20]:
df.corr()

,pclass,survived,sex,sibsp,parch,fare,embarked
pclass,1.000000,-0.312469,0.124617,0.060832,0.018322,-0.558740,0.185479
survived,-0.312469,1.000000,-0.528693,-0.027825,0.082660,0.244479,-0.175313
sex,0.124617,-0.528693,1.000000,-0.109609,-0.213125,-0.185744,0.097960
sibsp,0.060832,-0.027825,-0.109609,1.000000,0.373587,0.160388,0.065567
parch,0.018322,0.082660,-0.213125,0.373587,1.000000,0.221668,0.044772
fare,-0.558740,0.244479,-0.185744,0.160388,0.221668,1.000000,-0.238181
embarked,0.185479,-0.175313,0.097960,0.065567,0.044772,-0.238181,1.000000


In [21]:
df_corr = df.corr().survived
df_corr

pclass     -0.312469
survived    1.000000
sex        -0.528693
sibsp      -0.027825
parch       0.082660
fare        0.244479
embarked   -0.175313
Name: survived, dtype: float64

In [22]:
features_indepth = []
threshold = 0.2
cols = df.columns
index = 0
for value in df_corr:
    if abs(value) > threshold:
        features_indepth.append(cols[index])
    index += 1
features_indepth.remove('survived')
features_indepth

['pclass', 'sex', 'fare']

### Prepare the data

In [23]:
x = df[features_indepth]
y = df['survived']

### split the data into train and test

In [24]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=12345)

### Creating different models functions

#### Logistic regression model

In [25]:
def create_model_lg():
    from sklearn.linear_model import LogisticRegressionCV
    
    model = LogisticRegressionCV()
    model.fit(x_train, y_train)
    return model

#### KNN Model

In [26]:
def create_model_knn():
    from sklearn.neighbors import KNeighborsClassifier
    
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(x_train, y_train)
    return model

#### Decision Tree Model

In [27]:
def create_model_dt():
    from sklearn.tree import DecisionTreeClassifier
    
    model = DecisionTreeClassifier()
    model.fit(x_train, y_train)
    return model
    

#### SVM Model

In [28]:
def create_model_svm():
    from sklearn.svm import SVC
    
    model = SVC()
    model.fit(x_train, y_train)
    return model

#### Random Forest Model

In [29]:
def create_model_rf():
    from sklearn.ensemble import RandomForestClassifier
    
    model = RandomForestClassifier(n_estimators=500)
    model.fit(x_train, y_train)
    return model

#### XGBoost Model

In [30]:
def create_model_xgboost():
    import xgboost as xgb
    
    model = xgb.XGBClassifier()
    model.fit(x_train, y_train)
    return model

#### Creating the models

In [31]:
model_lg = create_model_lg()
model_knn = create_model_knn()
model_dt = create_model_dt()
model_svm = create_model_svm()
model_rf = create_model_rf()
model_xgb = create_model_xgboost()


### Evaluating the model

In [32]:
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_model(model, model_name):
    print(f"---Evaluation Report of {model_name}---")
    
    y_pred = model.predict(x_test)
    
     # get confusion matrix
    print(confusion_matrix(y_test, y_pred))
    
    # get the classification report
    print(classification_report(y_test, y_pred))

In [33]:
evaluate_model(model_lg, "Logistic Regression")
evaluate_model(model_knn, "KNN")
evaluate_model(model_dt, "Decision Tree")
evaluate_model(model_svm, "SVM")
evaluate_model(model_rf, "Random Forest")
evaluate_model(model_xgb, "XGBoost")



---Evaluation Report of Logistic Regression---
[[124  20]
 [ 35  83]]
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       144
           1       0.81      0.70      0.75       118

    accuracy                           0.79       262
   macro avg       0.79      0.78      0.78       262
weighted avg       0.79      0.79      0.79       262

---Evaluation Report of KNN---
[[122  22]
 [ 44  74]]
              precision    recall  f1-score   support

           0       0.73      0.85      0.79       144
           1       0.77      0.63      0.69       118

    accuracy                           0.75       262
   macro avg       0.75      0.74      0.74       262
weighted avg       0.75      0.75      0.74       262

---Evaluation Report of Decision Tree---
[[133  11]
 [ 41  77]]
              precision    recall  f1-score   support

           0       0.76      0.92      0.84       144
           1       0.88      0.65      0.75      